In [26]:
import sys
sys.path.append('/Projects/regionintelligenceai/')


In [27]:
import os
import requests
from datetime import datetime
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from src.driver_config import get_chrome_driver
from src.const import ORANGE_PLANNING_URL
from src.paths import RAW_DATA_DIR
import numpy as np


def retrieve_orange_city_pdf(directory=RAW_DATA_DIR):
    """
    Retrieves the Orange City PDF from the given URL and saves it in the specified directory.

    :param directory: Directory to save the PDF, defaults to RAW_DATA_DIR.
    :return: URL of the saved PDF or None if unsuccessful.
    """
    
    driver = get_chrome_driver()
    driver.get(ORANGE_PLANNING_URL)
    pdf_urls = []

    try:
        # Wait and click the accordion link
        accordion_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accordion-heading"))
        )
        accordion_link.click()

        # Wait and click the PDF link
        pdf_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "Current Pending Land Use Applications List"))
        )
        pdf_link.click()

        # Extract the PDF URL
        pdf_url = pdf_link.get_attribute("href")
        pdf_urls.append(pdf_url)
        
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()

    # Determine filename based on the current date
    current_date = datetime.now().strftime('%Y-%m-%d')
    file_name = f"orange_city_data_{current_date}.pdf"
   
    # Download and save the PDFs
    for i, url in enumerate(pdf_urls):
        response = requests.get(url)
        with open(os.path.join(directory / 'orange', file_name), "wb") as f:
            f.write(response.content)
    
    return pdf_urls[0] if pdf_urls else None

# Usage example
pdf_path = retrieve_orange_city_pdf()

In [28]:
def merge_rows(data):
    """
    Merges rows if the subsequent row is detected as a "leaked" row.
    A row is considered leaked if it contains more than a threshold number of None values.
    """
    merged_data = []
    previous_row = None

    def convert_empty_to_nan(row):
        """Convert empty strings in a row to NaN"""
        return [cell if cell != '' else np.nan for cell in row]

    for row in data:
        row = convert_empty_to_nan(row)
        if row.count(None) + row.count(np.nan) >= len(row) - 2:  # Adjust this criterion based on your data
            if previous_row:  # Merge with the previous row
                merged_row = []
                for prev_cell, cell in zip(previous_row, row):
                    if cell is not np.nan:
                        merged_row.append(f"{prev_cell} {cell}")
                    else:
                        merged_row.append(prev_cell)
                previous_row = merged_row
            else:
                previous_row = row
        else:
            if previous_row:
                merged_data.append(previous_row)
                previous_row = None
            merged_data.append(row)

    if previous_row:  # Append the last row if not appended
        merged_data.append(previous_row)

    return merged_data

In [29]:
from src.paths import RAW_DATA_DIR, PROCESSED_DATA_DIR
import pdfplumber
import pandas as pd
import glob


def parse_orange_pdf(pdf_path=(RAW_DATA_DIR / 'orange' / 'orange_city_data_2023-10-10.pdf')):
      # Open the PDF
    with pdfplumber.open(pdf_path) as pdf:
        all_data = []
        
        for page in pdf.pages:
            data = page.extract_table()
            
            # Since the structure is consistent, we can use indices to filter the data
            # You might need to adjust the indices based on the exact structure of the tables in the entire PDF
            filtered_data = [ 
            [row[1], row[3], row[4], row[5], ' '.join([str(cell) for cell in row[6:20] if cell is not None]), ' '.join([str(cell) for cell in row[22:27] if cell is not None])]
                for row in data[1:]  # Skipping the header
            ]            
            all_data.extend(filtered_data)
    # Convert the extracted data into a DataFrame
    columns = ['address', 'projectName', 'description', 'planner', 'status', 'recentUpdate/owner']
    df = pd.DataFrame(all_data, columns=columns)
    
    return df

df = parse_orange_pdf()
df
# row[6] through row[20] meed to be combined into one column

,address,projectName,description,planner,status,recentUpdate/owner
0,837 E Adams\nAve,Heidi Chen\nChew,Conversion of an\nexisting RV garage\ninto a t...,Tiffany Chhan,P\ne\nn\nd\nin\ng,7/20/2021 8/18/2021 Heidi Chew
1,1502 E\nAdams Ave,Navarrete\nMulti-\nFamily ADU,Multi-Family\ngarage conversion\ninto a 663 sq...,Vidal F. Márquez,A\np\np\nr\no\nv\ne\nd,9/27/2022 Oscar Vega
2,2089 N Agate\nSt,Ramirez\nResidence,A request to add\ntwo bedrooms\nand three\nbat...,Monique Schwartz,P\ne\nn\nd\nin\ng,9/28/2022 Antonio\nRamirez
3,None,None,None,None,,Comme
4,None,None,None,None,,nt
...,...,...,...,...,...,...
782,335 S\nWayfield St,Rosa ADU,Convert three\nattached garages\ninto one 550\...,Amber Gregg,P\ne\nn\nd\nin\ng,9/8/2023 Danny Cabrera theaduguys@gmail.com (...
783,7112 E\nWilderness\nAve,Mills ADU,"Proposed\nconstruction of a\nADU, 2 bed and 2\...",Tiffany Chhan,P\ne\nn\nd\nin\ng,5/10/2021 10/4/2022 Mark Mills mmills899@att.n...
784,1932 W\nWillow Ave,Genchev\nADU,New detached\n744 SF ADU,Ryan Agbayani,P\ne\nn\nd\nin\ng,1/23/2023 Jose Luigi\nSalemi salemijr@yahoo.c...
785,1932 W\nWillow Ave,Genchev\nADR,New 2nd story\naddition to an\nexisting 1 stor...,Ryan Agbayani,P\ne\nn\nd\nin\ng,1/23/2023 Jose Luigi\nSalemi salemijr@yahoo.c...


In [30]:
def clean_orange_pdf(df):
    # Convert all None values to NaN
    df = df.where(pd.notna(df), None)

    # Drop rows where at least 4 columns are NaN
    df = df[df.apply(lambda x: x.isna().sum() < 3, axis=1)]

    # Cleaning operations
    #df['recentUpdate/owner'] = df['recentUpdate/owner'].str.split().str[0]
    df['status'] = df['status'].str.replace('\n', '', regex=False)
    
    for column in df.columns:
        if column != 'status':
            df[column] = df[column].str.replace('\n', ' ', regex=False)

    return df

df = clean_orange_pdf(df)
df[:20]


,address,projectName,description,planner,status,recentUpdate/owner
0,837 E Adams Ave,Heidi Chen Chew,Conversion of an existing RV garage into a two...,Tiffany Chhan,Pending,7/20/2021 8/18/2021 Heidi Chew
1,1502 E Adams Ave,Navarrete Multi- Family ADU,Multi-Family garage conversion into a 663 squa...,Vidal F. Márquez,Approved,9/27/2022 Oscar Vega
2,2089 N Agate St,Ramirez Residence,A request to add two bedrooms and three bathro...,Monique Schwartz,Pending,9/28/2022 Antonio Ramirez
9,1115 E Alder Grove Cir,Magsanide ADU,"New 590 square foot attached one bedroom, one ...",Amber Gregg,,Comme 7/6/2023 Rodney Magsanide
14,171 S Anita Dr,Turner Healthcare,Exterior and interior modifications to an exis...,Ryan Agbayani,Pending,9/18/2023 Turner Healthcare Facilities Acquis...
16,553 S Arlington Rd,Millard Addition and ADU,New second story addition and convert existing...,Ryan Agbayani,,11/28/2 10/12/2022 Sondra Millard
26,413 E Barkley Ave,Stout ADU,Construct a new detached 633 SF ADU.,Ryan Agbayani,Pending,4/4/23 - 2/15/2023 4/4/2023 Roy Riveroy Jr
33,630 N Batavia St,Batavia Self Storage,Construct 3 new self-storage buildings totalin...,None,Vidal F. Márquez Pending Pending Pending,Comme nt letter sent 8/3/202 3 4/4/2022
34,2060 N Batavia St,FXI Warehouse,New warehouse building to replace fire damage ...,None,Vidal F. Márquez Pending Pending,Comme nt letter sent 8/3/202 3 12/22/2022
40,2425 N Batavia St,DISH Wireless,,Dish Wireless,Vidal F. Márquez Approved,3/1/2023 CDD Approve d 11/3/2022


In [31]:
import re
import pandas as pd

def extract_recent_date_and_owner(s):
    # Extract all dates in the format MM/DD/YYYY
    dates = re.findall(r'\d{1,2}/\d{1,2}/\d{4}', s)
    recent_date = None
    if dates:
        # Convert strings to datetime objects to find the most recent date
        recent_date = max(pd.to_datetime(dates)).strftime('%m/%d/%Y')
    
    # Assuming names are capitalized words. Extract them
    name_list = re.findall(r'\b[A-Z][a-z]+\b', s)
    owner = ' '.join(name_list[:2]) if name_list else None  # Considering first and last names only

    return recent_date, owner

def process_recentUpdate_owner_column(df):
    # Apply the extraction function to the 'recentUpdate/owner' column
    df['recentUpdate'], df['owner'] = zip(*df['recentUpdate/owner'].apply(extract_recent_date_and_owner))
    
    # Drop the original 'recentUpdate/owner' column
    df.drop(columns=['recentUpdate/owner'], inplace=True)
    
    return df

# Apply the function to your DataFrame
df = process_recentUpdate_owner_column(df)


In [32]:
df[:20]

,address,projectName,description,planner,status,recentUpdate,owner
0,837 E Adams Ave,Heidi Chen Chew,Conversion of an existing RV garage into a two...,Tiffany Chhan,Pending,08/18/2021,Heidi Chew
1,1502 E Adams Ave,Navarrete Multi- Family ADU,Multi-Family garage conversion into a 663 squa...,Vidal F. Márquez,Approved,09/27/2022,Oscar Vega
2,2089 N Agate St,Ramirez Residence,A request to add two bedrooms and three bathro...,Monique Schwartz,Pending,09/28/2022,Antonio Ramirez
9,1115 E Alder Grove Cir,Magsanide ADU,"New 590 square foot attached one bedroom, one ...",Amber Gregg,,07/06/2023,Comme Rodney
14,171 S Anita Dr,Turner Healthcare,Exterior and interior modifications to an exis...,Ryan Agbayani,Pending,09/18/2023,Turner Healthcare
16,553 S Arlington Rd,Millard Addition and ADU,New second story addition and convert existing...,Ryan Agbayani,,10/12/2022,Sondra Millard
26,413 E Barkley Ave,Stout ADU,Construct a new detached 633 SF ADU.,Ryan Agbayani,Pending,04/04/2023,Roy Riveroy
33,630 N Batavia St,Batavia Self Storage,Construct 3 new self-storage buildings totalin...,None,Vidal F. Márquez Pending Pending Pending,04/04/2022,Comme
34,2060 N Batavia St,FXI Warehouse,New warehouse building to replace fire damage ...,None,Vidal F. Márquez Pending Pending,12/22/2022,Comme
40,2425 N Batavia St,DISH Wireless,,Dish Wireless,Vidal F. Márquez Approved,03/01/2023,Approve


In [33]:
df1 = df

In [34]:
import re
from src.const import orange_planner_emails, orange_planner_phones, orange_planner_names

def move_text_from_status_to_planner(row):
    # Extract text other than "Pending" or "Approved"
    extraneous_text = re.sub(r'Pending|Approved', '', row['status']).strip()

    # If there's extraneous text and the planner is None, update the planner column
    if extraneous_text and (row['planner'] == 'None' or pd.isna(row['planner'])):
        row['planner'] = extraneous_text

    # Remove extraneous text from the status column
    row['status'] = row['status'].replace(extraneous_text, '').strip()

    # If status contains both "Pending" and "Approved", set it to "Pending"
    if "Approved" in row['status'] and "Pending" in row['status']:
        row['status'] = "Pending"
    elif "Pending" in row['status']:
        row['status'] = "Pending"
    elif "Approved" in row['status']:
        row['status'] = "Approved"
    else:
        row['status'] = "Denied"

    return row

def process_dataframe(df):
    df = df.apply(move_text_from_status_to_planner, axis=1)
    def extract_planner_name(row):
        names_set = set(orange_planner_names.values()) # Set of all planner names
        for name in names_set:
            if name in row:
                return name
        return None
    
    df['planner'] = df['planner'].apply(extract_planner_name)
    df['phone'] = df['planner'].map(orange_planner_phones)
    df['email'] = df['planner'].map(orange_planner_emails)
    return df

def update_description_based_on_project(row):
    # If 'ADU' is in projectName and description is empty or NaN
    if "ADU" in str(row['projectName']) and (pd.isnull(row['description']) or row['description'] == ''):
        row['description'] = "Applicant has requested to build an ADU"
    return row

# Apply the function to each row of the DataFrame
df = df.apply(update_description_based_on_project, axis=1)

# Drop rows where 'projectName' is either NaN, empty string, or only whitespace
df = df[df['projectName'].astype(str).str.strip() != '']
# Drop rows where 'projectName' is either NaN, empty string, or only whitespace
df = df[df['projectName'].astype(str).str.strip() != ' ']
# Drop rows where 'projectName' is empty or NaN
df = df.dropna(subset=['projectName'])

df = process_dataframe(df)
df


,address,projectName,description,planner,status,recentUpdate,owner,phone,email
0,837 E Adams Ave,Heidi Chen Chew,Conversion of an existing RV garage into a two...,Tiffany Chhan,Pending,08/18/2021,Heidi Chew,(714) 744-7272,tchhan@cityoforange.org
1,1502 E Adams Ave,Navarrete Multi- Family ADU,Multi-Family garage conversion into a 663 squa...,Vidal F. Márquez,Approved,09/27/2022,Oscar Vega,(714) 744-7241,vmarquez@cityoforange.org
2,2089 N Agate St,Ramirez Residence,A request to add two bedrooms and three bathro...,Monique Schwartz,Pending,09/28/2022,Antonio Ramirez,(714) 744-7220,mschwartz@cityoforange.org
9,1115 E Alder Grove Cir,Magsanide ADU,"New 590 square foot attached one bedroom, one ...",Amber Gregg,Denied,07/06/2023,Comme Rodney,(714) 744-7220,cdinfo@cityoforange.org
14,171 S Anita Dr,Turner Healthcare,Exterior and interior modifications to an exis...,Ryan Agbayani,Pending,09/18/2023,Turner Healthcare,(714) 744-7252,rabegglen@cityoforange.org
...,...,...,...,...,...,...,...,...,...
782,335 S Wayfield St,Rosa ADU,Convert three attached garages into one 550 sq...,Amber Gregg,Pending,09/08/2023,Danny Cabrera,(714) 744-7220,cdinfo@cityoforange.org
783,7112 E Wilderness Ave,Mills ADU,"Proposed construction of a ADU, 2 bed and 2 bath",Tiffany Chhan,Pending,10/04/2022,Mark Mills,(714) 744-7272,tchhan@cityoforange.org
784,1932 W Willow Ave,Genchev ADU,New detached 744 SF ADU,Ryan Agbayani,Pending,01/23/2023,Jose Luigi,(714) 744-7252,rabegglen@cityoforange.org
785,1932 W Willow Ave,Genchev ADR,New 2nd story addition to an existing 1 story ...,Ryan Agbayani,Pending,01/23/2023,Jose Luigi,(714) 744-7252,rabegglen@cityoforange.org


In [35]:
df[:20]

,address,projectName,description,planner,status,recentUpdate,owner,phone,email
0,837 E Adams Ave,Heidi Chen Chew,Conversion of an existing RV garage into a two...,Tiffany Chhan,Pending,08/18/2021,Heidi Chew,(714) 744-7272,tchhan@cityoforange.org
1,1502 E Adams Ave,Navarrete Multi- Family ADU,Multi-Family garage conversion into a 663 squa...,Vidal F. Márquez,Approved,09/27/2022,Oscar Vega,(714) 744-7241,vmarquez@cityoforange.org
2,2089 N Agate St,Ramirez Residence,A request to add two bedrooms and three bathro...,Monique Schwartz,Pending,09/28/2022,Antonio Ramirez,(714) 744-7220,mschwartz@cityoforange.org
9,1115 E Alder Grove Cir,Magsanide ADU,"New 590 square foot attached one bedroom, one ...",Amber Gregg,Denied,07/06/2023,Comme Rodney,(714) 744-7220,cdinfo@cityoforange.org
14,171 S Anita Dr,Turner Healthcare,Exterior and interior modifications to an exis...,Ryan Agbayani,Pending,09/18/2023,Turner Healthcare,(714) 744-7252,rabegglen@cityoforange.org
16,553 S Arlington Rd,Millard Addition and ADU,New second story addition and convert existing...,Ryan Agbayani,Denied,10/12/2022,Sondra Millard,(714) 744-7252,rabegglen@cityoforange.org
26,413 E Barkley Ave,Stout ADU,Construct a new detached 633 SF ADU.,Ryan Agbayani,Pending,04/04/2023,Roy Riveroy,(714) 744-7252,rabegglen@cityoforange.org
33,630 N Batavia St,Batavia Self Storage,Construct 3 new self-storage buildings totalin...,Vidal F. Márquez,Pending,04/04/2022,Comme,(714) 744-7241,vmarquez@cityoforange.org
34,2060 N Batavia St,FXI Warehouse,New warehouse building to replace fire damage ...,Vidal F. Márquez,Pending,12/22/2022,Comme,(714) 744-7241,vmarquez@cityoforange.org
40,2425 N Batavia St,DISH Wireless,,None,Approved,03/01/2023,Approve,NaN,NaN


In [46]:

class OrangeScraper:

    # Define current_date as a class variable
    current_date = datetime.now().strftime('%Y-%m-%d')

    def __init__(self, driver):
        self.driver = driver
        self.pdf_urls = []

    def connect(self, url):
        self.driver.get(url)
        print(self.driver.title)

    def scrape_pdf(self):
        try:
            accordion_link = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "accordion-heading"))
            )
            accordion_link.click()

            pdf_link = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.LINK_TEXT, "Current Pending Land Use Applications List"))
            )
            pdf_link.click()

            pdf_url = pdf_link.get_attribute("href")
            self.pdf_urls.append(pdf_url)
            
        except Exception as e:
            print(f"An error occurred: {e}")

    def parse_orange_pdf(self, pdf_path=None):
        if not pdf_path:
            pdf_path = RAW_DATA_DIR / 'orange' / f'orange_city_data_{OrangeScraper.current_date}.pdf'
            
        # Open the PDF
        with pdfplumber.open(pdf_path) as pdf:
            all_data = []
            
            for page in pdf.pages:
                data = page.extract_table()
                filtered_data = [ 
                    [row[1], row[3], row[4], row[5], ' '.join([str(cell) for cell in row[6:20] if cell is not None]), ' '.join([str(cell) for cell in row[22:27] if cell is not None])]
                    for row in data[1:]  # Skipping the header
                ]
                all_data.extend(filtered_data)

        # Convert the extracted data into a DataFrame
        columns = ['address', 'projectName', 'description', 'planner', 'status', 'recentUpdate/owner']
        df = pd.DataFrame(all_data, columns=columns)
        
        return df

    def download_pdf(self, directory):
        if not self.pdf_urls:
            print("No PDFs found to download!")
            return

        # Use the class variable here
        file_name = f"orange_city_data_{OrangeScraper.current_date}.pdf"

        orange_dir = os.path.join(directory, 'orange')
        if not os.path.exists(orange_dir):
            os.makedirs(orange_dir)

        for url in self.pdf_urls:
            response = requests.get(url)
            with open(os.path.join(orange_dir, file_name), "wb") as f:
                f.write(response.content)
        
        return os.path.join(orange_dir, file_name)
    
    def clean_orange_pdf(self, df):
        # Convert all None values to NaN
        df = df.where(pd.notna(df), None)

        # Drop rows where at least 4 columns are NaN
        df = df[df.apply(lambda x: x.isna().sum() < 3, axis=1)]

        # Cleaning operations
        df['status'] = df['status'].str.replace('\n', '', regex=False)
        
        for column in df.columns:
            if column != 'status':
                df[column] = df[column].str.replace('\n', ' ', regex=False)

        return df
    
    def move_text_from_status_to_planner(self, row):
        # Extract text other than "Pending" or "Approved"
        extraneous_text = re.sub(r'Pending|Approved', '', row['status']).strip()

        # If there's extraneous text and the planner is None, update the planner column
        if extraneous_text and (row['planner'] == 'None' or pd.isna(row['planner'])):
            row['planner'] = extraneous_text

        # Remove extraneous text from the status column
        row['status'] = row['status'].replace(extraneous_text, '').strip()

        # If status contains both "Pending" and "Approved", set it to "Pending"
        if "Approved" in row['status'] and "Pending" in row['status']:
            row['status'] = "Pending"
        elif "Pending" in row['status']:
            row['status'] = "Pending"
        elif "Approved" in row['status']:
            row['status'] = "Approved"
        else:
            row['status'] = "Denied"

        return row
    
    def process_dataframe(self, df):
        df = df.apply(self.move_text_from_status_to_planner, axis=1)
        def extract_planner_name(row):
            names_set = set(orange_planner_names.values()) # Set of all planner names
            for name in names_set:
                if name in row:
                    return name
            return None
        
        df['planner'] = df['planner'].apply(extract_planner_name)
        df['phone'] = df['planner'].map(orange_planner_phones)
        df['email'] = df['planner'].map(orange_planner_emails)
        return df
    
    def update_description_based_on_project(self, row):
        # If 'ADU' is in projectName and description is empty or NaN
        if "ADU" in str(row['projectName']) and (pd.isnull(row['description']) or row['description'] == ''):
            row['description'] = "Applicant has requested to build an ADU"
        return row

    def refine_dataframe(self, df):
        df = df.apply(self.update_description_based_on_project, axis=1)
        df = df[df['projectName'].astype(str).str.strip() != '']
        df = df[df['projectName'].astype(str).str.strip() != ' ']
        df = df.dropna(subset=['projectName'])
        return df
    
    def save_to_processed(self, df):
        # Use the class variable here
        file_name = f"orange_city_data_{OrangeScraper.current_date}.xlsx"
        path = PROCESSED_DATA_DIR / 'orange' / file_name
        df.to_excel(path, header=True)
    
    def extract_recent_date_and_owner(self, s):
        # Extract all dates in the format MM/DD/YYYY
        dates = re.findall(r'\d{1,2}/\d{1,2}/\d{4}', s)
        recent_date = None
        if dates:
            # Convert strings to datetime objects to find the most recent date
            recent_date = max(pd.to_datetime(dates)).strftime('%m/%d/%Y')
    
        # Assuming names are capitalized words. Extract them
        name_list = re.findall(r'\b[A-Z][a-z]+\b', s)
        owner = ' '.join(name_list[:2]) if name_list else None  # Considering first and last names only

        return recent_date, owner
    
    def process_recentUpdate_owner_column(self, df):
        # Apply the extraction function to the 'recentUpdate/owner' column
        df['recentUpdate'], df['owner'] = zip(*df['recentUpdate/owner'].apply(self.extract_recent_date_and_owner))
        
        # Drop the original 'recentUpdate/owner' column
        df.drop(columns=['recentUpdate/owner'], inplace=True)
        
        df.reset_index(drop=True, inplace=True)

        return df

# Example usage
driver = get_chrome_driver()
scraper = OrangeScraper(driver)
scraper.connect(ORANGE_PLANNING_URL)
scraper.scrape_pdf()
pdf_path = scraper.download_pdf(RAW_DATA_DIR)
df = scraper.parse_orange_pdf(pdf_path)
df = scraper.clean_orange_pdf(df)
df = scraper.refine_dataframe(df)
df = scraper.process_dataframe(df)
df = scraper.process_recentUpdate_owner_column(df)
scraper.save_to_processed(df)
driver.close()

Current Projects | City of Orange, CA
                     address  \
0            837 E Adams Ave   
1           1502 E Adams Ave   
2            2089 N Agate St   
3     1115 E Alder Grove Cir   
4             171 S Anita Dr   
5         553 S Arlington Rd   
6          413 E Barkley Ave   
7           630 N Batavia St   
8          2060 N Batavia St   
9          2425 N Batavia St   
10        3126 E Bradbury Ct   
11        7102 E Cambria Cir   
12       1550 N Cambridge St   
13            1502 Cannon St   
14           267 N Center St   
15       415 S Center Street   
16  1415 & 1417 E Century Dr   
17         1415 E Century Dr   
18         208 E Chapman Ave   
19         212 E Chapman Ave   

                                          projectName  \
0                                     Heidi Chen Chew   
1                         Navarrete Multi- Family ADU   
2                                   Ramirez Residence   
3                                       Magsanide ADU   
4   